### LDA Topics clustering with Gensim

In [18]:
import gensim
from gensim import corpora, models

from modules.nlputils import StemFilterTokenizeProcessor

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
proc = StemFilterTokenizeProcessor(stopwords=stopwords, min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)
tokens = proc.tokenize_documents(speeches)

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary, passes=100)
print(lda_model)
lda_model.print_topics(4)

LdaModel(num_terms=3763, num_topics=4, decay=0.5, chunksize=2000)


[(0,
  '0.016*"gente" + 0.008*"população" + 0.008*"mato" + 0.007*"empresas" + 0.007*"grosso" + 0.007*"setor" + 0.007*"pessoa" + 0.007*"ministro" + 0.007*"dinheiro" + 0.007*"projeto"'),
 (1,
  '0.009*"resistência" + 0.009*"preço" + 0.008*"projeto" + 0.008*"saúde" + 0.007*"população" + 0.007*"violência" + 0.007*"mercado" + 0.007*"trabalho" + 0.006*"mundo" + 0.006*"rio"'),
 (2,
  '0.017*"trabalhadores" + 0.015*"rio" + 0.013*"projeto" + 0.013*"sul" + 0.010*"região" + 0.008*"veto" + 0.008*"votação" + 0.008*"mundo" + 0.007*"direitos" + 0.007*"comissão"'),
 (3,
  '0.015*"tribunal" + 0.011*"ministros" + 0.009*"ordem" + 0.009*"sessão" + 0.008*"processo" + 0.007*"república" + 0.007*"regimento" + 0.007*"ministro" + 0.007*"decisão" + 0.007*"plenário"')]

### LDA Topics clustering with Scikitlearn

In [23]:
import modules.scikitutils as scikitutils
from modules.nlputils import StemFilterTokenizeProcessor
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]
print('All speeches 06/2018')
print('qtty=' + str(len(speeches)))

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
# stopwords = []
proc = StemFilterTokenizeProcessor(min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)

tf_vectorizer = CountVectorizer(min_df=0.1, max_df=1.0, stop_words=stopwords, preprocessor=proc.process_text)
tf = tf_vectorizer.fit_transform(speeches)
tf_feature_names = tf_vectorizer.get_feature_names()

lda = LatentDirichletAllocation(n_topics=4, max_iter=200, learning_method='batch', random_state=0, n_jobs=2)
lda.fit(tf)

scikitutils.display_lda_topics(lda, tf_feature_names, 5)

All speeches 06/2018
qtty=466
Topic 0: trabalhadores preço mercado resistência bilhões 
Topic 1: rio trabalho sul violência brasileiros 
Topic 2: gente ministro ministério direito tribunal 
Topic 3: projeto sessão ordem câmara deputado 


### Clusterize with KMeans

In [22]:
import pandas as pd
from modules.nlputils import StemFilterTokenizeProcessor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans, KMeans

speeches_df = pd.read_csv('output/speeches/senate-speeches-201806.csv', index_col='CodigoPronunciamento')
speeches = speeches_df[['TextoIntegral']]
speeches = [s[0] for s in speeches.values[:]]

stopwords = set([line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8')])
proc = StemFilterTokenizeProcessor(min_size=3, filter_regex='[a-z]', stem_language='portuguese', stem_complete=True, only_nouns=True)

vec = TfidfVectorizer(min_df=0.1, stop_words=stopwords, analyzer='word', ngram_range=(1, 2), preprocessor=proc.process_text)
X = vec.fit_transform(speeches)

number_of_clusters = 4
kmeans_model = KMeans(n_clusters=number_of_clusters)
speech_groups = kmeans_model.fit_predict(X)

order_centroids = kmeans_model.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
cluster_labels = []
for i in range(number_of_clusters):
    top_words = [terms[ind] for ind in order_centroids[i, :5]]
    gl = "Cluster {}: {}".format(i, ' '.join(top_words))
    print(gl)
    cluster_labels.append(gl)

results = pd.DataFrame()
results['speeches'] = speeches
results['category'] = kmeans_model.labels_

for index, row in results[:20].iterrows():
    print('========')
    print(str(row['category']) + ' ' + cluster_labels[row['category']] + ' ' + row['speeches'])


Cluster 0: veto rio rio sul sul região
Cluster 1: ordem votação projeto sessão deputado
Cluster 2: preço petróleo gasolina empresa setor
Cluster 3: brasileiros mundo resistência gente ministro
2 Cluster 2: preço petróleo gasolina empresa setor      O SR. ROBERTO REQUIÃO  (Bloco Maioria/MDB - PR. Pronuncia o seguinte discurso. Sem revisão do orador.) – Senador Presidente, Senador Paim, vou aproveitar esta sexta-feira para repetir o mantra que eu venho repetindo com insistência sobre o que realmente está acontecendo no Brasil. Nós não estamos, Senador Paim, descolados de uma circunstância global. O que acontece no Brasil, acontece em outros países também.     O Papa Francisco, quando os governantes mais poderosos do mundo se reuniam em Davos, com os homens mais ricos da Terra, os concentradores do capital, dizia: "O capital é bom quando ele é investido, gera empregos, produtos, inova na tecnologia, mas o capital tem que servir à sociedade e não pode dominá-la." Na verdade, o que o Papa e

,speeches,category
0,O SR. ROBERTO REQUIÃO (Bloco Maioria/MDB...,2
1,O SR. PAULO PAIM (Bloco Parlamentar da R...,2
2,O SR. CRISTOVAM BUARQUE (Bloco Parlament...,2
3,O SR. REGUFFE (S/Partido - DF. Pronuncia...,3
4,O SR. HÉLIO JOSÉ (Bloco Maioria/PROS - D...,2
5,O SR. JOSÉ MEDEIROS (Bloco Parlamentar D...,3
6,A SRª VANESSA GRAZZIOTIN (Bloco Parlamen...,3
7,O SR. HUMBERTO COSTA (Bloco Parlamentar ...,3
8,O SR. ACIR GURGACZ (Bloco Parlamentar da...,3
9,O SR. PAULO PAIM (Bloco Parlamentar da R...,2
